## PuLPモジュール読み込み

In [1]:
import pulp

## 点の集合，枝の集合，容量を定義

In [2]:
V = [i+1 for i in range(5)]
E = [(1,2), (1,3), (2,1), (2,3), (2,5),  (3,4), (3,5), (4,5), (5,3)]
U = {(1,2):4, (1,3):3, (2,1):1, (2,3):4, (2,5):3, (3,4):3, (3,5):2, (4,5):2, (5,3):2}
s = 1
t = 5

## 決定変数の定義

In [3]:
x = {e:pulp.LpVariable(f'x{e[0]}_{e[1]}', lowBound=0, upBound=U[e]) for e in E} # upBound=U[e] 容量制約
z = pulp.LpVariable('z')

## 問題の定義

In [4]:
p = pulp.LpProblem('最大流問題', sense=pulp.LpMaximize)
p += z, '目的関数　流量'

In [5]:
# 流量保存制約
for i in V:
    inflow = []
    outflow = []
    for e in E:
        if e[1] == i:
            inflow.append(e)
        if e[0] == i:
            outflow.append(e)
    net_outflow = pulp.lpSum(x[e] for e in outflow) - pulp.lpSum(x[e] for e in inflow)
    if i == s:
        p += net_outflow == z, f'始点{i}'
    elif i == t:
        p += net_outflow == -z, f'終点{i}'
    else:
        p += net_outflow == 0, f'点{i}'
p

最大流問題:
MAXIMIZE
1*z + 0
SUBJECT TO
始点1: x1_2 + x1_3 - x2_1 - z = 0

点2: - x1_2 + x2_1 + x2_3 + x2_5 = 0

点3: - x1_3 - x2_3 + x3_4 + x3_5 - x5_3 = 0

点4: - x3_4 + x4_5 = 0

終点5: - x2_5 - x3_5 - x4_5 + x5_3 + z = 0

VARIABLES
x1_2 <= 4 Continuous
x1_3 <= 3 Continuous
x2_1 <= 1 Continuous
x2_3 <= 4 Continuous
x2_5 <= 3 Continuous
x3_4 <= 3 Continuous
x3_5 <= 2 Continuous
x4_5 <= 2 Continuous
x5_3 <= 2 Continuous
z free Continuous

## 最適解の計算と計算結果の読み取り

In [6]:
result = p.solve()

In [7]:
pulp.LpStatus[result]

'Optimal'

In [8]:
pulp.value(p.objective)

7.0

In [9]:
for v in p.variables():
    if pulp.value(v) > 0:
        print(f'{v} = {pulp.value(v)}')

x1_2 = 4.0
x1_3 = 3.0
x2_3 = 1.0
x2_5 = 3.0
x3_4 = 2.0
x3_5 = 2.0
x4_5 = 2.0
z = 7.0
